In [1]:
#Importing libraries
import re
import pickle
import pandas as pd
import psycopg2

In [2]:
#Setting up the paths and filenames
path = r'D:\\Projects\\ipums\\garrard_test\\'
dataname = r'ky_garrard_1870_88v_test_usa_00006.dat'
cbkname = r'ky_1870_cbk.pickle'
sliceobjs = r'ky_1870_slice_objects.pickle'
prefix = 'cn1870_'

slice_objs = path + sliceobjs
cbkfile = path + cbkname
datafile = path + dataname

In [3]:
#Pickle functions
def load_pickle(filename, objectname):
        with open(path+filename, "rb") as infile:
            objectname = pickle.load(infile)
        print('Loaded from:', filename)

def save_pickle(filename, objectname):
    with open(path+filename, "wb") as outfile:
        pickle.dump(objectname, outfile)
    print('Saved to:',filename)

In [4]:
#loading the codebook and creating the lists of columns and slices
cbk = pd.read_pickle(cbkfile)
cbk.columns
x = cbk['length'].astype('str')
cbk['column_name'] = prefix+cbk['variable'].str.lower() + ' char('+x+')'
column_list = cbk['column_name'].tolist()
slice_list = cbk['slice_obj'].tolist()
print('column_list:\n',column_list[0:10])
print('slice_list:\n',slice_list[0:10])

column_list:
 ['cn1870_year char(4)', 'cn1870_serial char(8)', 'cn1870_numprec char(2)', 'cn1870_subsamp char(2)', 'cn1870_numperhh char(4)', 'cn1870_hhtype char(1)', 'cn1870_region char(2)', 'cn1870_stateicp char(2)', 'cn1870_statefip char(2)', 'cn1870_countyicp char(4)']
slice_list:
 [slice(0, 4, None), slice(4, 12, None), slice(12, 14, None), slice(14, 16, None), slice(16, 20, None), slice(20, 21, None), slice(21, 23, None), slice(23, 25, None), slice(25, 27, None), slice(27, 31, None)]


In [5]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cur.execute("SET SCHEMA 'census'")

def close_database():
    conn.commit() #insert rollback logic
    conn.close()

def create_table(tablename, column_list):
    columns_text = '(' + 'id serial PRIMARY KEY, ' + ' '.join(column_list) + ');'
    print(columns)
    cur.execute('DROP TABLE IF EXISTS ' + tablename + ';')
    cur.execute('CREATE TABLE ' + tablename + ' ' + columns_text)
    cur.execute('SELECT * FROM ' + tablename + ';') #to confirm table is created with correct column names

def get_column_count(tablename, columns):
    qry_cc = 
    '''
    select count(*) from information_schema.columns
    where table_name='<table_name>’;
    '''
    col_ct_sql = cur.execute(qry_cc)
    col_ct_list = len(columns)+1
    print('columns in table:',col_ct_sql,'\n','columns in list:',col_ct_list)
    
def process_line(line, slices):
    limit = len(line)
    data = []
    for i in slices:
        len_slice = len(line[i])
        begin = limit
        limit = limit - len_slice
        if limit <= 0: break
        data.append(line[i])
    return data

def insert_tuple(tablename, data):
    query = "insert into " + tablename + " values %s returning *"
    cur.execute(query, (data,)) # notice the comma after the tuple
    #query = """
    #    insert into table values %s
    #    returning *
    #"""
    

In [7]:
#Open the test database and define a cursor  
conn = open_database('test')
cur = create_cursor()
set_census_schema()

In [ ]:
#create_table('census_1870',col_list)
#get_column_counts('census_1870,col_list)


In [ ]:
records = 10
rec_count = 0
with open(datafile) as f:
    for i in range(records):
        line = f.readline()
        if line == "": break
        data = tuple(process_line(line, slice_list))
        #insert_tuple('census', data)
        rec_count += 1
        ##print('processed',rec_count,'record')
        #print(data)
print('Total records processed:', rec_count)

Total records processed: 10


#### Opening a connection
Connect to an existing database
>>> conn = psycopg2.connect("dbname=test user=postgres")

#### Open a cursor to perform database operations
>>> cur = conn.cursor()

#### Execute a command: this creates a new table
>>> cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")


#### Inserting the tuples as a single value

query = """
    insert into t values %s
    returning *
"""
my_tuple = (2, 'b')

cursor.execute(query, (my_tuple,)) # Notice the comma after my_tuple
rs = cursor.fetchall()
conn.commit()
for row in rs:
    print row